In [6]:
# Libraries
import numpy as np    

# Definitions
A = np.array([[401.,-201.],[-800.,401.]])
b = np.array([200.,-200.])
M = A + 0.01*np.identity(b.shape[0])

In [8]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        error_i = np.linalg.norm(b - np.dot(A, np.dot(Q[:,:i+1], y)))
        
        if H[i+1,i] == 0 or error_i < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [ ]:
# Optimized version of GMRes, pendent
def opt_gmres(A,b,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        error_i = np.linalg.norm(b - np.dot(A, np.dot(Q[:,:i+1], y)))
        
        if H[i+1,i] == 0 or error_i < tol:
            break
    
    return np.dot(Q[:,:-1], y)

In [2]:
# GMres with preconditioner proposed by the paper. Bad implementation
# TODO: BORRAR EL M^-1, EL PROFE NOS MATARA!
def paper_bad_prec_gmres(A,b,M,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    Z = np.zeros((b.shape[0], it+1))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    M_1 = np.linalg.inv(M)
    i = 0
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        Z[:,i] = np.dot(M_1,Q[:,i])
        w = np.dot(A,Z[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        error_i = np.linalg.norm(b - np.dot(A, np.dot(Z[:,:i+1], y)))
        
        if H[i+1,i] == 0 or error_i < tol:
            break
    return np.dot(Z[:,:i+1], y),i

In [3]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b,it=it,tol=tol)[0]
    for i in range(b.shape[0]):
        A[:,i] = gmres(M, A[:,i],it=it,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        error_i = np.linalg.norm(b - np.dot(A, np.dot(Q[:,:i+1], y)))
        
        if H[i+1,i] == 0 or error_i < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [4]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A, gmres(M, Q[:,i],it=it,tol=tol)[0])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        # Use the residual error from leastsquares to avoid another GMRes computation
        if H[i+1,i] == 0 or np.linalg.norm(residual)/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    return gmres(M, x_tild,it=it,tol=tol)[0], i

In [9]:
testcases = ['normal','cancer','left','right']
test = testcases[2]
error = 1e-18

if test == 'normal':
    print gmres(A,b,tol=error)
elif test == 'cancer':
    print paper_bad_prec_gmres(A,b,M)
elif test == 'right':
    print right_prec_gmres(A,b,M,tol=error)
elif test == 'left':
    print left_prec_gmres(A,b,M,tol=error)

(array([ 40000.0000049 ,  79800.00000978]), 93)
